# 실행 전 requirements.txt 터미널에서 받아주세요!

### 버스 노선 데이터 노선별 이용량 추출
교통카드빅데이터 통합 정보 시스템: https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC03&indiSel=IC0301


In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
busroute_usage = pd.read_csv(
    filepath_or_buffer="../data/gyeongn_busroute_total.csv"
    )

In [3]:
busroute_usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2513 entries, 0 to 2512
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   city_name         2513 non-null   object 
 1   city_code         2513 non-null   int64  
 2   routeid           2513 non-null   object 
 3   routeno           2513 non-null   object 
 4   routetp           2513 non-null   object 
 5   startnodenm       2513 non-null   object 
 6   endnodenm         2513 non-null   object 
 7   startvehicletime  1413 non-null   float64
 8   endvehicletime    1334 non-null   float64
 9   intervaltime      1056 non-null   float64
 10  intervalsattime   159 non-null    float64
 11  intervalsuntime   152 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 235.7+ KB


In [4]:
busroute_usage["city_name"].unique()

array(['창원시', '진주시', '통영시', '사천시', '김해시', '밀양시', '거제시', '양산시', '의령군',
       '함안군', '창녕군', '고성군', '남해군', '하동군', '산청군', '함양군', '거창군', '합천군'],
      dtype=object)

In [5]:
busroute_usage["city_name"].value_counts()

city_name
진주시    300
거제시    279
통영시    216
함안군    209
밀양시    189
창원시    164
합천군    149
거창군    141
양산시    115
창녕군    115
고성군    112
김해시     98
남해군     89
의령군     74
함양군     73
산청군     66
하동군     64
사천시     60
Name: count, dtype: int64

#### 웹에서 파일 다운로드하는 크롤러 만들기

In [6]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from tqdm import tqdm

In [7]:
## 도시명 입력
city_name = "의령군" 
input_values = busroute_usage[busroute_usage["city_name"] == city_name][["routeno", "startnodenm"]] 
findtime = 100000

route_usage_data = {}

if city_name == "창원시":
    region_button_selector = "48123_48127_48125_48120_48121_48129"
    
elif city_name == "진주시":
    region_button_selector = "48170"
    
elif city_name == "통영시":
    region_button_selector = "48220"
    
elif city_name == "사천시":
    region_button_selector = "48240"
    
elif city_name == "김해시":
    region_button_selector = "48250"
    
elif city_name == "밀양시":
    region_button_selector = "48270"
        
elif city_name == "거제시":
    region_button_selector = "48310"
    
elif city_name == "양산시":
    region_button_selector = "48330"
    
elif city_name == "의령군":
    region_button_selector = "48720"
    
elif city_name == "함안군":
    region_button_selector = "48730"
    
elif city_name == "창녕군":
    region_button_selector = "48740"
    
elif city_name == "고성군":
    region_button_selector = "48820"
    
elif city_name == "남해군":
    region_button_selector = "48840"
    
elif city_name == "하동군":
    region_button_selector = "48850"
    
elif city_name == "산청군":
    region_button_selector = "48860"
    
elif city_name == "함양군":
    region_button_selector = "48870"

elif city_name == "거창군":
    region_button_selector = "48880"
    
elif city_name == "합천군":
    region_button_selector = "48890"
    
else:
    raise ValueError(f"Unknown city_name: {city_name}")

### 반복문 시작 #########################################################################################
route_usage_data[f"{city_name}"] = {}
first_i = input_values.index[0] + 1 

for i, input_value in tqdm(input_values.iterrows()):
    routeno = input_value["routeno"]
    startnodenm = input_value["startnodenm"]
    print(f"    ## Start Extract:{i+1}")
    print(f"        routeno    : {routeno}")
    print(f"        startnodenm: {startnodenm}")

    options = Options()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    driver = wb.Chrome(options = options)

    ## 교통카드빅데이터 종합정보시스템 -> 노선별 이용량 조회 url
    driver.get("https://stcis.go.kr/pivotIndi/wpsPivotIndicator.do?siteGb=P&indiClss=IC03&indiSel=IC0301")

    ## 클릭 함수
    def driver_clickable_processor(css_selector_value):
        click = WebDriverWait(driver, findtime).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector_value))
        )    
        driver.execute_script("arguments[0].scrollIntoView();", click)
        driver.execute_script("arguments[0].click();", click)    
        
        return click

    ## 텍스트 추출 함수 
    def driver_extract_text_processor(css_selector_value):
        element = WebDriverWait(driver, findtime).until(
                        EC.presence_of_element_located((
                            By.CSS_SELECTOR, 
                            css_selector_value
                            ))
                    )
        return element.text

    ## 기간선택 -> 월 클릭
    driver_clickable_processor("#divRdoDate > li:nth-child(2)")

    ## 공간선택 -> 검색명에 city_name 입력 후 검색 클릭
    click = driver_clickable_processor("#searchRouteSpaceNm")
    click.send_keys(f"{city_name}")

    ## 검색 버튼 클릭
    click = driver_clickable_processor("#space4 > li.box_flex > button")

    ## 시도 박스 클릭
    dropdown = Select(driver.find_element(By.XPATH, '//*[@id="searchPopZoneSd"]'))
    dropdown.select_by_index(14)

    ## 지역 버튼 클릭 > 지역마다 클릭 버튼이 다름
    dropdown = Select(driver.find_element(By.XPATH, '//*[@id="searchPopZoneSgg"]'))
    dropdown.select_by_value(region_button_selector)
    
    ## 노선명 입력칸 클릭 -> 노선번호(명) 입력
    print("        Search Node")
    route_name_input = driver_clickable_processor("#popupSearchRouteNo")
    route_name_input.send_keys(f"{routeno}")

    ## 검색 버튼 클릭
    search_button = driver_clickable_processor("#route_space1 > li.box_flex > button")

    ## 노선명과 기종점이 같은지 확인
    routeno_len = int(WebDriverWait(driver, findtime).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'pop_box'))
    ).text.split()[2])

    try:
        WebDriverWait(driver, findtime).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#divBusLineList > div.pop_box > table > tbody"))
        )

        ## 검색결과 한개인 경우
        if routeno_len == 1:
            check_routeno = driver.find_element(
                By.CSS_SELECTOR, 
                "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(2)"
                ).text

            check_startnodem = driver.find_element(
                By.CSS_SELECTOR, 
                "#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(4)"
                ).text.split(" - ")[0]

            ## 노선명과 기종점이 일치할 경우 체크 박스 클릭
            if check_routeno == str(routeno): ## and check_startnodem == startnodenm
                check_click = driver_clickable_processor("#divBusLineList > div.pop_box > table > tbody > tr > td:nth-child(1) > div > label > div")
                select_button = driver_clickable_processor("#popupBusLine > div:nth-child(4) > button")
                
                ## 첫번째만 실행 이용자유형, 시간대 체크 표시 없애기
                # if i == first_i:
                type_no_check = driver_clickable_processor("#divChkDmn > li:nth-child(8) > label > div")            
                time_no_check = driver_clickable_processor("#divChkDmn > li:nth-child(9) > label > div")
            
                ## 검색결과조회 버튼 클릭
                search_click = driver_clickable_processor("#btnSearch > button")
                            
                ## 경고 메세지 "확인" 클릭
                try:
                    WebDriverWait(driver, findtime).until(
                        EC.alert_is_present()
                    )
                    alert = driver.switch_to.alert
                    alert.accept()  # '확인' 클릭
                except Exception as e:
                    print(f"팝업 처리 오류: {e}")
                
                ## 노선 이용량 추출
                print("        Start Extraction1")
                route_usage_element = driver_extract_text_processor("#pivotGrid > div > div:nth-child(2) > div.wj-cells > div:nth-child(2) > div")
                route_usage_int = int(route_usage_element.replace(",", ""))
                print("        Done Extraction1")
                
        ## 검색결과 여러 개인 경우
        else:         
            for j in range(1, 11):
                check_routeno = driver.find_element(
                    By.CSS_SELECTOR, 
                    f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({j}) > td:nth-child(2)"
                    ).text

                check_startnodem = driver.find_element(
                    By.CSS_SELECTOR, 
                    f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({j}) > td:nth-child(4)"
                    ).text.split(" - ")[0]

                ## 노선명과 기종점이 일치할 경우 체크 박스 클릭
                if check_routeno == str(routeno): ## and check_startnodem == startnodenm:
                    driver_clickable_processor(f"#divBusLineList > div.pop_box > table > tbody > tr:nth-child({j}) > td:nth-child(1) > div > label > div")
            
                # for-loop 중단
                break
            
            # 선택버튼 클릭
            select_button = driver_clickable_processor("#popupBusLine > div:nth-child(4) > button")
                    
            ## 첫번째만 실행 이용자유형, 시간대 체크 표시 없애기
            # if i == first_i:
            type_no_check = driver_clickable_processor("#divChkDmn > li:nth-child(8) > label > div")            
            time_no_check = driver_clickable_processor("#divChkDmn > li:nth-child(9) > label > div")

            ## 검색결과조회 버튼 클릭
            search_click = driver_clickable_processor("#btnSearch > button")
            
            ## 경고 메세지 "확인" 클릭
            try:
                WebDriverWait(driver, findtime).until(
                    EC.alert_is_present()
                )
                alert = driver.switch_to.alert
                alert.accept()  # '확인' 클릭
            except Exception as e:
                print(f"팝업 처리 오류: {e}")
            
            ## 노선 이용량 추출
            print("        Start Extraction")
            route_usage_element = driver_extract_text_processor("#pivotGrid > div > div:nth-child(2) > div.wj-cells > div:nth-child(2) > div")
            route_usage_int = int(route_usage_element.replace(",", ""))
            print("        Done Extraction")
            
        print(f"        Save Usage : {route_usage_int}")
        route_usage_data[f"{city_name}"][f"{routeno}"] = route_usage_int
        
        ## 창닫기
        driver.quit()
 
    except:
        ## 빈칸 넣기
        route_usage_data[f"{city_name}"][f"{routeno}"] = None
        
        ## 창닫기
        print("         Error")           
        driver.quit()

0it [00:00, ?it/s]

    ## Start Extract:1422
        routeno    : 101-1
        startnodenm: 의령버스터미널
        Search Node
         Error


1it [00:23, 23.88s/it]

    ## Start Extract:1423
        routeno    : 101-3
        startnodenm: 의령버스터미널
        Search Node
         Error


2it [01:03, 32.93s/it]

...

    ## Start Extract:1437
        routeno    : 2-37
        startnodenm: 의령버스터미널
        Search Node
         Error


16it [07:23, 25.97s/it]

    ## Start Extract:1438
        routeno    : 2-37
        startnodenm: 다현마을


16it [07:40, 28.78s/it]


KeyboardInterrupt: 

In [18]:
route_usage_data

{'산청군': {'5': None,
  '11': None,
  '55': None,
  '21': None,
  '51': None,
  '4': None,
  '62': None,
  '48': None,
  '50': None,
  '13': None,
  '40': 959,
  '60': None,
  '14': 959,
  '6': None,
  '12': None,
  '26': None,
  '23': None,
  '53': None,
  '32': None,
  '54': None,
  '2': 959,
  '25': None,
  '10': None,
  '24': None,
  '1': 8796,
  '74': None,
  '56': None,
  '8': None,
  '72': None,
  '22': None,
  '20': None,
  '70': None,
  '33': None,
  '66': None,
  '7': None,
  '64': None,
  '41': None,
  '57': None,
  '80': None,
  '93': None,
  '65': None,
  '94': None,
  '61': None,
  '95': None,
  '82': None,
  '67': None,
  '84': None,
  '68': None,
  '85': None,
  '31': None,
  '71': None,
  '91': None,
  '34': 8796,
  '73': None,
  '90': None,
  '3': 8796,
  '81': None,
  '49': None,
  '52': None,
  '47': None,
  '75': None,
  '83': None,
  '63': None,
  '59': None,
  '58': None,
  '92': None}}

### 결측값 채우기

In [14]:
route_usage_data

{'사천시': {'100': 793,
  '101': 1385,
  '102': 1085,
  '110': 15598,
  '111': 2309,
  '112': 537,
  '130': 1545,
  '132': 541,
  '132-1': None,
  '140': 892,
  '141': 831,
  '142': 2074,
  '143': 636,
  '144': 290,
  '313': 292,
  '190': 604,
  '190-1': 555,
  '191': 383,
  '191-1': 385,
  '192': 236,
  '193': 284,
  '193-1': 161,
  '194': 4323,
  '200': 217,
  '200-1': 332,
  '201': 193,
  '201-1': 294,
  '220': 2866,
  '221': 607,
  '230': 5016,
  '230-1': 4455,
  '310': 409,
  '311': 411,
  '260': 38,
  '192-1': 293,
  '160': 531,
  '380': 85,
  '161': 252,
  '370': 148,
  '316': 709,
  '206': 83,
  '312': 178,
  '266': 85,
  '113': 6300,
  '104': 880,
  '105': 1260,
  '106': 317,
  '145': 41,
  '209': 878,
  '229': 1119,
  '103': 61,
  '222': 130,
  '223': 328,
  '228': 814,
  '363': 154,
  '500': 1573,
  '501': 959}}

### pickle 저장

In [19]:
import pickle

with open(f'../data/버스이용데이터/{city_name}_usage.pkl', 'wb') as f: 
	pickle.dump(route_usage_data, f)